Na dzisiejszych zajęciach będziemy zajmować się algorytmami drzewiastymi. Poznamy dwa algorytmy z biblioteki scikit-learn: random forest (lasy losowe) oraz extra trees.   

Oprócz tego zrobimy krótkie wprowadzenie do pakietu LightGBM wykorzystującego metodę wzmocnienia gradientowego.

Dane, na których będziemy pracować dotyczą tzw. klasyfikacji zdarzeń akustycznych (ang. acoustic event classification). Są to różnego rodzaju dźwięki, które można spotkać w mieście - głosy ludzi, zwierząt, odgłosy wydawane przez urządzenia techniczne, samochody itp. Jest to dość trudny problem klasyfikacyjny, ponieważ próbujemy nauczyć model rozpoznawania wielu klas naraz.

Źrodło danych: https://www.kaggle.com/chrisfilo/urbansound8k?fbclid=IwAR2AwTcNOEFFJGq3Me5C2o6lLJ_jMTzo5PUdB2EWStjTaBq1z1CVCCQG0Uc

In [1]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import roc_auc_score, roc_curve, auc, precision_recall_curve, accuracy_score, recall_score, f1_score, make_scorer, confusion_matrix, log_loss
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.svm import SVC
import optuna
from sklearn.model_selection import cross_validate, GroupKFold
import pickle
from collections import Counter
import lightgbm as lgb
import scipy.stats

c:\Users\jakub\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Tak jak ostatnio, będziemy pracować na plikach z wyekstrahowanymi wcześniej cechami, żeby nie tracić czasu na ich wyliczanie. Będziemy wykorzystywać dwa rodzaje cech: poznane już wcześniej MFCC oraz bardzo podobne cechy, których jeszcze nie omawialiśmy - GFCC (ang. gammatone frequency cepstral coefficients).

GFCC to kolejny rodzaj cech bazujący na modelu psychoakustycznym, który ma za zadanie opisać w sposób matematyczny działanie ludzkiego układu słuchowego. Od MFCC różni się przede wszystkich kształtem filtrów stosowanych do przetwarzania sygnałów - w przypadku MFCC są to filtry trójkątne, natomiast GFCC wyliczane są przy użyciu filtrów gammatone. Filtry gammatone mają nieco bardziej naturalny, fizjologiczny kształt - wierzchołek jest zaokrąglony, a nie ostry jak w filtrach trójkątnych:

![caption](https://www.researchgate.net/profile/Peter-Balazs/publication/309663739/figure/fig2/AS:614042511085584@1523410551779/A-popular-auditory-filter-model-the-gammatone-filter-bank-The-magnitude-responses-in.png)

Klasyfikację przeprowadzimy na początek używając tylko MFCC i lasów losowych. Wczytujemy 3 pliki:
- mfcc_feats.npy - macierz z 13 MFCC
- labels.npy - wektor z labelami (przyporządkowaniem do klas)
- folds.npy - podział na foldy (podzbiory), który ma być wykorzystany podczas walidacji krzyżowej. 

Dotychczas walidację krzyżową robiliśmy dzieląc dane na podzbiory w sposób losowy - czasem podział jest istotny i powinien być przeprowadzony w określony sposób. Jest tak np. wtedy, gdy w bazie znajduje się więcej niż jeden sygnał zawierający dźwięki emitowane przez to samo urządzenie lub tę samą osobę. Wtedy należy zagwarantować, żeby  wszystkie sygnały związane z tym samym obiektem znalazły się w tym samym podzbiorze - gdyby tak nie było, to uzyskalibyśmy sztucznie zawyżone metryki i mielibyśmy złe pojęcie o jakości naszego modelu.

In [2]:
X_mfcc = np.load('lab7/mfcc_3D.npy')
y_org = np.load('lab7/labels.npy')
folds = np.load('lab7/folds.npy')
#klasy
# 0 = airconditioner 1 = carhorn 2 = childrenplaying 3 = dogbark 4 = drilling
# 5 = engineidling 6 = gunshot 7 = jackhammer 8 = siren 9 = street_music

X_mfcc.shape #mamy 7334 sygnałów o długości 345 ramek, z każdego wyekstrahowano 13 MFCC

(7334, 13, 345)

Gdybyśmy utworzyli z macierzy MFCC wektor, mielibyśmy każdy sygnał opisany 4486 cechami - algorytmy drzewiaste uczyłyby się na takich danych bardzo długo. Dlatego chcemy mieć dane o mniejszych wymiarach.

Tym razem nie będziemy zmniejszać wymiarowości danych używając algorytmów, które poznaliśmy do tej pory. Zamiast tego wyliczymy parametry statystyczne charakteryzujące każdy kanał (każdy z 13 współczynników cepstralnych) na całej długości sygnału. Będą to bardzo podstawowe parametry:
- wartość średnia, 
- odchylenie standardowe, 
- mediana, 
- I i III kwartyl, 
- rozrzut pomiędzy 10 i 90 percentylem, 
- kurtoza, 
- skośność, 
- wartość minimalna,
- wartość maksymalna. 

Takie cechy są czasem stosowane w przemyśle, np. w analizie mowy, więc warto wiedzieć, że można ich używać i że potrafią dawać dobre rezultaty.

Teraz każdy sygnał będzie opisany tylko 130 cechami, więc można prowadzić proces uczenia, walidacji i predykcji bez dalszej redukcji wymiarowości.

In [3]:
X = np.hstack((np.mean(X_mfcc, axis=2), 
               np.std(X_mfcc, axis=2), 
               np.median(X_mfcc, axis=2), 
               np.percentile(X_mfcc, 25, axis=2), 
               np.percentile(X_mfcc, 75, axis=2), 
               scipy.stats.iqr(X_mfcc, rng=(10, 90), axis=2),
               scipy.stats.kurtosis(X_mfcc, axis=2),
               scipy.stats.skew(X_mfcc, axis=2),
               np.min(X_mfcc, axis=2),
               np.max(X_mfcc, axis=2)
              ))

Wyświetlmy liczebność poszczególnych klas, żeby zobaczyć, czy mają podobną liczebność - jeżeli różnica pomiędzy liczebnością klas będzie bardzo duża, to prawdopodobnie klasyfikator nie będzie w stanie nauczyć się ich poprawnie rozpoznawać bez dodatkowego preprocessingu, np. nad- lub podpróbkowania klas.

In [4]:
Counter(y_org)

Counter({2: 976,
         3: 675,
         0: 997,
         9: 1000,
         8: 897,
         5: 961,
         7: 804,
         4: 805,
         1: 203,
         6: 16})

Jak widać najmniej liczba jest klasa 6 - należy do niej tylko 16 obiektów, czyli 0,2% wszystkich danych. Usuniemy wszystkie obiekty z tej klasy ze zbioru danych, ponieważ będą tylko utrudniać proces uczenia.

Wykorzystamy podzbiór nr 10 jako zbiór testowy, a pozostałe 9 podzbiorów będzie tworzyło zbiór uczący. Walidacja krzyżowa, którą będziemy robić za chwilę, będzie w takim razie 9-krotna.

In [5]:
#usuwanie klasy 6 ze zbioru danych
y_not_6 = y_org!=6
print(folds.shape)

X = X[y_not_6]
folds = folds[y_not_6]
y = y_org[y_not_6]

#podział na zbiór uczący i testowy
train_folds_mask = folds != 10
test_fold_mask = folds == 10

train_folds = folds[train_folds_mask]
test_fold = folds[test_fold_mask]

X_train = X[train_folds_mask]
X_test = X[test_fold_mask]

y_train = y[train_folds_mask]
y_test = y[test_fold_mask]

print(train_folds)

(7334,)


Ostatnim krokiem przed uczeniem modelu będzie standaryzacja danych:

In [6]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

LAS LOSOWY

Mamy już przygotowane dane, więc możemy zdefiniować, która metryka posłuży do walidacji modelu w czasie walidacji krzyżowej oraz przejść do uczenia modelu. Zaczniemy od lasu losowego, czyli klasyfikatora Random Forest.

Do wyjaśnienia zasady działania lasów losowych potrzebna jest znajomość drzew decyzyjnych. Drzewa decyzyjne to hierachiczny algorytm, który tworzy proste reguły dotyczące wartości cech, które pozwalają dokonać podziału danych na klasy (lub przewidywania wartości w przypadku regresji). Początek drzewa, czyli miejsce, w którym dane będą po raz pierwszy podzielone nazywany jest korzeniem. Każde kolejne miejsce, znajdujące się co najmniej poziom niżej od korzenia, to węzeł. Na końcu drzewa znajdują się liście - w liściach nie następują już dalsze podziały, ponieważ albo znajdują się w nich obiekty z tylko jednej klasy, albo został osiągnięty narzucony warunek zatrzymania algorytmu. Węzły połączone są ze sobą oraz z liśćmi krawędziami. 

W węzłach zapisane są warunki dalszego podziału, dotyczące pojedynczych cech lub ich kombinacji liniowych. Predykcja polega na przejściu przez wszystkie poziomy drzewa, sprawdzając, czy spełnione są zapisane w kolejnych węzłach warunki. Na końcu docieramy do liścia, w którym zawarta jest informacja, do której klasy przynależy obiekt.

Poniżej przykład drzewa decyzyjnego służącego do określenia, czy u pacjenta występuje podwyższone ryzyko zawału mięśnia sercowego:

![caption](https://pranav-ap.netlify.app/media/machine%20learning/decision%20trees/decision-tree.png)

Trochę bardziej wyczerpujący opis można przeczytac np. tu: http://aiway.pl/drzewa-decyzyjne/

Las losowy to zbiór drzew klasyfikacyjnych. Każde drzewo uczone jest na innym, losowym podzbiorze danych - w ten sposób każde drzewo może uczyć się na innym zestawie obiektów. Dodatkowo, w każdym węźle drzewa podział dokonywany jest w oparciu o losowo wybrane cechy (czyli nie bierzemy pod uwagę wszystkich cech opisujących obiekt, a jedynie część z nich). Ostateczna przynależność do klas określana jest na drodze głosowania - każde drzewo zwraca decyzję, a cały las wybiera tę klasę, która pojawiła się najwięcej razy w predykcjach pojedynczych drzew.

In [7]:
scoring = {'f1_macro': make_scorer(f1_score, average='macro')}

Jeżeli mamy z góry określony podział na podzbiory, którego mamy użyć podczas walidacji krzyżowej, funkcji cross_validate należy użyć w taki sposób:
- najpierw podajemy model, który będzie uczony,
- następnie macierz cech uczących oraz wektor labeli zbioru uczącego,
- określamy, jaka metryka będzie wykorzystywana do walidacji, czyli scoring,
- definiujemy parametr groups - jest to podział obiektów na podzbiory określony w zmiennej train_folds (dla każdego obiektu podany jest numer podzbioru, w którym ma się znaleźć),
- do parametry cv przypisujemy GroupKFold(n_splits=len(np.unique(train_folds))) - to jest to miejsce, w którym algorytm walidacji krzyżowej sprawdza, ile podzbiorów ma być utworzonych i dostaje informację, że ma ten podział zrobić na podstawie numerów podzbiorów zawartych w train_folds.

In [8]:
model = RandomForestClassifier(random_state=42)
scores = cross_validate(model, X_train, y_train, scoring=scoring, groups=train_folds, cv=GroupKFold(n_splits=len(np.unique(train_folds))))

Wyświetlmy sobie, jakie wartości F1 uzyskaliśmy dla każdego podzbioru podczas walidacji krzyżowej.

In [9]:
scores

{'fit_time': array([13.96679282, 10.96430492, 11.15365911,  9.43510866,  9.02480412,
         9.6541481 ,  9.86483312, 10.10695052, 13.18539429]),
 'score_time': array([0.05573964, 0.09884977, 0.06856418, 0.05012226, 0.04794526,
        0.06979537, 0.06677055, 0.12436891, 0.07514787]),
 'test_f1_macro': array([0.6351108 , 0.46780824, 0.59506957, 0.50502088, 0.56636187,
        0.59618002, 0.57521537, 0.57631282, 0.54417924])}

Jak widać, wyniki nie są szczególnie dobre - dla 8 z 9 podzbiorów uzyskaliśmy F1 < 0.6. Jeżeli teraz przeprowadzimy walidację prostą (czyli uczenie na całym zbiorze uczącym i predykcje na testowym), to raczej też nie uzyskamy zadowalających wyników.

In [10]:
#Przeprowadź trening klasyfikatora RandomForest z random_state=42 i domyślnymi wartościami pozostałych
#hiperparamterów oraz predykcję na zbiorze testowy. 
#Wyświetl macierz pomyłek, F1 i dokładność (accuracy)
model = RandomForestClassifier(random_state=42)
model.fit(X_train,y_train)
test_preds = model.predict(X_test)
print(confusion_matrix(y_test, test_preds))
print(accuracy_score(y_test, test_preds))


[[50  0  7  0  1 33  0  0  9]
 [ 0  0 11  0  0  3  0  1  1]
 [ 1  0 73  6  0  2  0  3 13]
 [ 0  0 12 42  0  0  0  4 10]
 [ 0  0  4  0 47 12  5 12  4]
 [ 8  0  0  0  0 65  6  0 10]
 [ 0  0  0  0 26  0 47  0  0]
 [ 0  0 37  1  3  0  1 35  0]
 [ 0  0 14  0  2  0  1  2 81]]
0.624113475177305


Możemy również wyświetlić, które cechy miały największy wpływ na wynik klasyfikacji.

In [11]:
model.feature_importances_

array([0.01347559, 0.01289423, 0.0089694 , 0.0161301 , 0.00939055,
       0.00598816, 0.00843287, 0.00871002, 0.01048525, 0.00659015,
       0.00818963, 0.00719654, 0.00606685, 0.01578409, 0.01063738,
       0.00917164, 0.00927458, 0.01003863, 0.00670233, 0.00834102,
       0.00693825, 0.00565679, 0.00898747, 0.01707306, 0.00743833,
       0.00694824, 0.01290899, 0.01173283, 0.00904746, 0.01418193,
       0.00904204, 0.00573058, 0.00862992, 0.00790943, 0.00958022,
       0.00760183, 0.00683907, 0.00605683, 0.00523785, 0.01172233,
       0.01124313, 0.00911116, 0.01507182, 0.01249784, 0.00612842,
       0.00799496, 0.00721569, 0.00766232, 0.0071994 , 0.0083708 ,
       0.00682377, 0.00518819, 0.01275915, 0.01298717, 0.01020599,
       0.01522327, 0.00799157, 0.00572296, 0.00856745, 0.01128347,
       0.01250434, 0.00661601, 0.00850692, 0.00784823, 0.00684716,
       0.01481661, 0.01488154, 0.011767  , 0.0096766 , 0.0087968 ,
       0.00529765, 0.00913274, 0.00938902, 0.00433273, 0.00900

Możemy znaleźć indeksy cech, które są najbardziej i najmniej istotne:

In [12]:
np.argmax(model.feature_importances_), np.argmin(model.feature_importances_)

(23, 102)

Największy wpływ na wynik klasyfikacji miała cecha nr 23, a najmniejszy - 102. Patrząc na sposób tworzenia macierzy X możemy wywnioskować, że najistotniejsze jest odchylenie standardowe 10 kanału MFCC, zaś najmniej istotna - skośność 11 kanału.

Robiąc takie analizy moglibyśmy usunąć te najmniej istotne cechy i w ten sposób przeprowadzić redukcję wymiarowości. Nie będziemy tego teraz robić, ponieważ chcemy porównać wyniki uzyskiwane na różnych zbiorach cech oraz przy użyciu różnych klasyfikatorów.

Przypuszczenia oparte o wyniki dla podzbiorów walidacyjnych potwiedziły się i wyniki uzyskane na zbiorze testowym nie są zbyt dobre, chociaż też nie można powiedzieć, że są tragiczne - dokładność 0.62 dla modelu z domyślnymi wartościami hiperparametrów i przy 9 klasach to całkiem dobry wyniki. Widać natomiast, że np. żaden obiekt z klasy 2 nie zostal zaklasyfikowany prawidłowo, więc warto spróbować uzyskać lepszy model.

Możemy wypróbować kolejne cechy, czyli GFCC. Podziału na zbiory i labeli nie musimy wczytywać na nowo, bo pozostają bez zmian.

In [13]:
print(len(y)) #długość wektora y po usunięciu obiektów z klasy 6
X_gfcc = np.load('lab7/gfcc_3D.npy')
print(len(y)) #długość wektora y przed usunięciem obiektów z klasy 6
X_gfcc.shape

7318
7318


(7334, 13, 345)

Kolejne etapy są takie same - należy wyliczyć statystyki, usunąć klasę 6 z macierzy X i podzielić dane na zbiór uczący i testowy tak, by w testowym był tylko 10 podzbiór, a w uczącym pozostałe 9 podzbiorów.

Zrób to poniżej, nie nadpisując starego kodu - będziemy porównywać uzyskane wyniki, więc dobrze będzie mieć dostęp do tego, co wyszło przed chwilą.

In [14]:
#wylicz statystyki
X_gfcc_params = np.hstack((np.mean(X_gfcc, axis=2), 
               np.std(X_gfcc, axis=2), 
               np.median(X_gfcc, axis=2), 
               np.percentile(X_gfcc, 25, axis=2), 
               np.percentile(X_gfcc, 75, axis=2), 
               scipy.stats.iqr(X_gfcc, rng=(10, 90), axis=2),
               scipy.stats.kurtosis(X_gfcc, axis=2),
               scipy.stats.skew(X_gfcc, axis=2),
               np.min(X_gfcc, axis=2),
               np.max(X_gfcc, axis=2)
              ))

In [15]:
#usuń klasę 6 i podziel dane na zbiory
#usuwanie klasy 6 ze zbioru danych
print(folds.shape)

X_gfcc_params = X_gfcc_params[y_not_6]

X_train_gfcc = X_gfcc_params[train_folds_mask]
X_test_gfcc = X_gfcc_params[test_fold_mask]

scaler = StandardScaler()
X_train_gfcc = scaler.fit_transform(X_train_gfcc)
X_test_gfcc = scaler.transform(X_test_gfcc)

(7318,)


In [16]:
#przeprowadź walidację krzyżową klasyfikatora RandomForest (zastosuj taki sam random_state jak wcześniej)
model = RandomForestClassifier(random_state=42)
scores_gfcc = cross_validate(model, X_train_gfcc, y_train, scoring=scoring, groups=train_folds, cv=GroupKFold(n_splits=len(np.unique(train_folds))))

In [17]:
scores_gfcc

{'fit_time': array([18.66571116, 13.86148214, 11.8635776 , 11.42606711, 11.59590411,
        11.54210591, 13.43760777, 13.63634539, 13.73947668]),
 'score_time': array([0.06210637, 0.05098963, 0.03612876, 0.04559302, 0.0818572 ,
        0.04013205, 0.03224707, 0.05351329, 0.09697938]),
 'test_f1_macro': array([0.47118627, 0.38431533, 0.48526217, 0.41145696, 0.41658662,
        0.4106849 , 0.42931284, 0.47413933, 0.44017885])}

In [18]:
#Przeprowadź trening klasyfikatora RandomForest z random_state=42 i domyślnymi wartościami pozostałych
#hiperparamterów oraz predykcję na zbiorze testowy. 
#Wyświetl macierz pomyłek, F1 i dokładność (accuracy)
clf_gfcc = RandomForestClassifier(random_state=42)
clf_gfcc.fit(X_train_gfcc, y_train)
gfcc_preds = clf_gfcc.predict(X_test_gfcc)
print(confusion_matrix(y_test, gfcc_preds))
print(accuracy_score(y_test, gfcc_preds))


[[69  0 13  0  3 11  3  1  0]
 [ 1  0  4  1  5  0  1  0  4]
 [11  0 62  2  1  5  5  0 12]
 [ 2  0  8 44  0  1  0  1 12]
 [16  0  6  5 33  5  5  4 10]
 [19  0  5  0  0 43 11  2  9]
 [ 0  0  2  0  2  6 48  0 15]
 [ 9  0 22  1  3  2  0 31  9]
 [ 4  0 23  1 11  2  6  7 46]]
0.5333333333333333


Tym razem wyniki są znacząco gorsze niż te uzyskane na MFCC. 

Spróbujemy zwiększyć dokładność predykcji ucząc model na obu typach cech naraz. Takie postępowanie, czyli wykorzystywanie zbioru różnych cech, które pojedynczo nie dają dobrych rezultatów, często pozwala znacząco poprawić jakość modelu.

In [19]:
X_both = np.hstack((X_mfcc, X_gfcc))

X_both.shape

(7334, 26, 345)

In [20]:
#Ponownie wylicz statystyki
X_both_params = np.hstack((np.mean(X_both, axis=2), 
               np.std(X_both, axis=2), 
               np.median(X_both, axis=2), 
               np.percentile(X_both, 25, axis=2), 
               np.percentile(X_both, 75, axis=2), 
               scipy.stats.iqr(X_both, rng=(10, 90), axis=2),
               scipy.stats.kurtosis(X_both, axis=2),
               scipy.stats.skew(X_both, axis=2),
               np.min(X_both, axis=2),
               np.max(X_both, axis=2)
              ))

In [21]:
X_both_params.shape

(7334, 260)

Cech opisujących pojedynczy obiekt mamy teraz co prawda aż 260, ale nadal nie jest to na tyle duża liczba, żeby proces uczenia trwał bardzo długo lub było duże ryzyko przeuczenia modelu, więc możemy nie robić redukcji wymiarowości.

In [22]:
#usuń klasę 6, podziel dane na zbiory i przeprowadź walidację krzyżową, trening i predykcję
X_both_params = X_both_params[y_not_6]

X_train_both = X_both_params[train_folds_mask]
X_test_both = X_both_params[test_fold_mask]

In [23]:
scaler = StandardScaler()
X_train_both = scaler.fit_transform(X_train_both)
X_test_both = scaler.transform(X_test_both)

In [24]:
clf_both = RandomForestClassifier(random_state=42)
clf_both.fit(X_train_both, y_train)
both_preds = clf_both.predict(X_test_both)
print(confusion_matrix(y_test, both_preds))
print(accuracy_score(y_test,both_preds))

[[50  0 11  0  2 31  1  0  5]
 [ 0  1 10  0  0  2  0  0  3]
 [ 0  0 80  2  0  1  0  3 12]
 [ 1  0  9 50  0  0  0  1  7]
 [ 3  0 10  2 39 20  1  4  5]
 [10  0  2  0  4 59  3  1 10]
 [ 0  0  0  0 10  0 62  0  1]
 [ 0  1 35  0  1  0  0 39  1]
 [ 1  1 12  1  2  0  0  1 82]]
0.6553191489361702


Wyniki, które uzyskaliśmy na MFCC i GFCC równocześnie są już bardziej obiecujące - nawet na nieoptymalizowanym modelu udało uzyskać się dokładność i F1 przekraczające 0.65.

Skoro już wiemy, że na takich danych uzyskujemy najlepsze wyniki, możemy przejść do optymalizacji modelu. Będziemy optymalizować 3 hiperparamtry:
- n_estimators - liczba drzew klasyfikacyjnych, które będą tworzyły las,
- max_depth - maksymalna głębokość (liczba poziomów) drzewa,
- min_samples_split - minimalna liczba obiektów, które muszą znajdować się w węźle, by można go było poddać kolejnemu podziałowi.

Dodamy też parametr n_jobs, który określna liczbę rdzeni, które zostaną wykorzytane do treningu. -1 oznacza wszystkie dostępne - taki wybór przyspieszy optymalizację, ale spowolni działanie innych procesów.

Liczbę triali ustawimy na 60 - jest to wartość dobrana eksperymentalnie pozwalająca na uzyskanie w miarę dobrych wyników. W domu lub pod koniec zajęć możesz spróbować zwiększyć ją np. do 100 lub 150 i zobaczyć, czy wyniki uda się poprawić, czy optimum będzie jednak osiągnięte wcześniej i nie ma sensu optymalizować modelu dłużej.

In [28]:
model = RandomForestClassifier

#uzupełnij funkcję get_space
def get_space(trial): 
    space = {"n_estimators": trial.suggest_int("n_estimators", 10, 200),
        "max_depth": trial.suggest_int("max_depth", 1, 20),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "n_jobs": trial.suggest_int("n_jobs", -1, -1)}
    return space
trials = 3 #liczba prób

def objective(trial, model, X, y):
    model_space = get_space(trial)

    mdl = model(**model_space)
    scores = cross_validate(mdl, X, y, groups=train_folds, scoring=scoring, cv=GroupKFold(n_splits=len(np.unique(train_folds))), return_train_score=True)

    return np.mean(scores['test_f1_macro'])

In [29]:
study = optuna.create_study(direction='maximize')

[I 2022-12-19 20:18:35,385] A new study created in memory with name: no-name-e1cecff3-4f7a-44b2-8ce1-1d666134d997


In [30]:
%%time
#%%time to komenda służąca do pomiaru czasu wykonywania komórki, musi znajdować się na samej górze komórki
study.optimize(lambda x: objective(x, model, X_train, y_train), n_trials=trials)


[I 2022-12-19 20:19:55,852] Trial 0 finished with value: 0.5475451866834719 and parameters: {'n_estimators': 189, 'max_depth': 19, 'min_samples_split': 18, 'n_jobs': -1}. Best is trial 0 with value: 0.5475451866834719.
[I 2022-12-19 20:20:07,292] Trial 1 finished with value: 0.4995501158510172 and parameters: {'n_estimators': 31, 'max_depth': 7, 'min_samples_split': 4, 'n_jobs': -1}. Best is trial 0 with value: 0.5475451866834719.
[I 2022-12-19 20:20:35,950] Trial 2 finished with value: 0.5195347130730005 and parameters: {'n_estimators': 105, 'max_depth': 8, 'min_samples_split': 16, 'n_jobs': -1}. Best is trial 0 with value: 0.5475451866834719.


CPU times: total: 18.2 s
Wall time: 1min 58s


In [62]:
params = study.best_params
print(params)

{'n_estimators': 189, 'max_depth': 19, 'min_samples_split': 18, 'n_jobs': -1}


In [63]:
#Wytrenuj model o najlepszych hiperparametrach, przeprowadź predykcję i wylicz metryki

#clf_both_best = RandomForestClassifier(random_state=42, n_estimators=params[0], max_depth=params[1], min_samples_split=params[2], n_jobs=params[3])
clf_both_best = RandomForestClassifier(random_state=42, **params)
clf_both_best.fit(X_train_both, y_train)
both_preds_best = clf_both_best.predict(X_test_both)
print(confusion_matrix(y_test, both_preds_best))
print(accuracy_score(y_test,both_preds_best))

[[52  0 10  0  1 31  0  1  5]
 [ 0  0  6  0  2  2  1  0  5]
 [ 0  0 84  2  0  2  0  1  9]
 [ 2  0 11 45  0  0  0  1  9]
 [ 0  0  5  3 43 24  0  5  4]
 [ 7  0  1  0  1 70  1  0  9]
 [ 1  0  0  0  4  0 68  0  0]
 [ 0  0 37  0  2  0  0 36  2]
 [ 0  0 12  1  2  0  0  3 82]]
0.6808510638297872


Czy wyniki uzyskane przy użyciu optymalizowanego modelu są lepsze, niż te, które uzyskaliśmy stosując domyślne wartości hiperparametrów?

EXTRA TREES

Algorytmem bardzo podobnym do lasu losowego jest algorytm ExtraTrees. Od lasu losowego różni go przede wszystkich kryterium podziału drzewa w danym węźle: w ExtraTrees kryterium jest losowe, a w lasach losowych stosowane jest kryterium optymalne (dające najlepszy podział). Dzięki temu algorytm ExtraTrees zazwyczaj jest szybszy. Kolejną istotną różnicą jest to, że podczas podziału danych w lesie losowym stosowane jest tzw. losowanie ze zwracaniem, natomiast w ExtraTrees nie - oznacza to, że w ExtraTrees obserwacje podane do kolejnych drzew nie będą się powtarzać, natomiast w lesie losowym mogą.

ExtraTrees najczęściej daje (trochę) gorsze wyniki niż las losowy, jednak nie zawsze tak jest - jeżeli mamy czas, warto wypróbować oba algorytmy.

In [64]:
model = ExtraTreesClassifier

#uzupełnij funkcję get_space
def get_space(trial): 
    space = {"n_estimators": trial.suggest_int("n_estimators", 10, 200),
        "max_depth": trial.suggest_int("max_depth", 1, 20),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "n_jobs": trial.suggest_int("n_jobs", -1, -1)}
    return space
trials = 3 #liczba prób

def objective(trial, model, X, y):
    model_space = get_space(trial)

    mdl = model(**model_space)
    scores = cross_validate(mdl, X, y, groups=train_folds, scoring=scoring, cv=GroupKFold(n_splits=len(np.unique(train_folds))), return_train_score=True)

    return np.mean(scores['test_f1_macro'])

In [65]:
study = optuna.create_study(direction='maximize')

[I 2022-12-19 20:35:17,837] A new study created in memory with name: no-name-a9ce7fcd-1297-48e6-91ef-ef0ac165ebdb


In [66]:
%%time
study.optimize(lambda x: objective(x, model, X_train, y_train), n_trials=trials)

[I 2022-12-19 20:35:38,003] Trial 0 finished with value: 0.5384231945959664 and parameters: {'n_estimators': 46, 'max_depth': 15, 'min_samples_split': 3, 'n_jobs': -1}. Best is trial 0 with value: 0.5384231945959664.
[I 2022-12-19 20:35:53,876] Trial 1 finished with value: 0.5572924272271176 and parameters: {'n_estimators': 84, 'max_depth': 18, 'min_samples_split': 10, 'n_jobs': -1}. Best is trial 1 with value: 0.5572924272271176.
[I 2022-12-19 20:36:08,651] Trial 2 finished with value: 0.46449491834946394 and parameters: {'n_estimators': 78, 'max_depth': 4, 'min_samples_split': 12, 'n_jobs': -1}. Best is trial 1 with value: 0.5572924272271176.


CPU times: total: 20.2 s
Wall time: 48.4 s


In [67]:
#Wytrenuj model o najlepszych hiperparametrach, przeprowadź predykcję i wylicz metryki
params = study.best_params
print(params)
clf_both_best = ExtraTreesClassifier(random_state=42, **params)
clf_both_best.fit(X_train_both, y_train)
both_preds_best = clf_both_best.predict(X_test_both)
print(confusion_matrix(y_test, both_preds_best))
print(accuracy_score(y_test,both_preds_best))

{'n_estimators': 84, 'max_depth': 18, 'min_samples_split': 10, 'n_jobs': -1}
[[52  0  7  0  2 31  1  0  7]
 [ 0  0  8  0  0  2  0  0  6]
 [ 0  0 84  3  0  1  0  1  9]
 [ 1  0 13 45  0  0  0  2  7]
 [ 2  0 12  1 54  5  1  3  6]
 [13  0  3  0  0 50 15  0  8]
 [ 1  0  0  0 11  0 61  0  0]
 [ 0  0 36  0  1  0  3 36  1]
 [ 0  0 12  0  0  0  1  3 84]]
0.6609929078014184


LightGBM

Ostatnim klasyfikatorem przewidzianym na dzisiaj jest algorytm drzewiasty zaimplementowany w pakiecie LightGBM. W porównaniu do lasów losowych i ExtraTrees cechuje się przede wszystkim większą efektywnością, mniejszym zużyciem pamięci i często pozwala uzyskać większą dokładność klasyfikacji. Ma też o wiele więcej hiperparametrów, które możemy określić lub optymalizować: https://lightgbm.readthedocs.io/en/latest/Parameters.html

My będziemy optymalizować następujące z nich:
- boosting_type - algorytm wzmocnienia gradientowego, zostawimy wartość domyślną, czyli gbdt (Gradient Boosting Decision Tree),
- num_leaves - maksymalna liczba liści, które mogą zostać utworzone,
- n_estimators - liczba drzew,
- max_depth - maksymalna głębokość drzewa (liczba poziomów),
- learning_rate - współczynnik określający, z jakim krokiem mają być zmieniane wagi modelu podczas dążenia do osiągnięcia minimalnego błędu popełnianego przez model (zbyt mały learning rate powoduje nadmiernie długi czas uczenia, zbyt duży zwiększa ryzyko "przeskoczenia" minimum błędu i nieznalezienia optymalnych wag),
- subsample - określa, jaka część obiektów (domyślnie losowanych bez zwracania) ma być podana do drzewa podczas treningu, 
- colsample_bytree - określa, jaka część wszystkich cech ma być podana do drzewa podczas treningu, 
- min_child_samples - minimalna liczba obiektów, które muszą znaleźć się w liściu,
- min_split_gain - minimalny przyrost informacji, który musi być osiągnięty po podziale węzła (jeżeli nie jest osiągnięty, to węzeł nie będzie dalej dzielony); przyrost informacji definiowany jest jako entropia_przed_podziałem - entropia_po_podziale, gdzie entropia dana jest wzorem:

![caption](https://lh6.googleusercontent.com/B4y6fIw_pcAR-AWuTlF5mVlmvKximSRUDhptG-KhvcNLD_RAoITCSi7fn4Owjufm9pnpV3BFrdcijPtA2vCqtSHIPz0Ydrr5NqY924XV-oSl5eOUqfluX9isYXQoU3WHL55x9eLH)

In [68]:
model = lgb.LGBMClassifier
def get_space(trial): 
    space = {"boosting_type": trial.suggest_categorical('boosting_type', ["gbdt"]),
            "num_leaves": trial.suggest_int("num_leaves", 2, 200),
            "n_estimators": trial.suggest_int("n_estimators", 2, 200),
            "max_depth": trial.suggest_int("max_depth", 2, 200),
            "learning_rate": trial.suggest_discrete_uniform("learning_rate", 0.00001, 1, 0.0001),
            "subsample": trial.suggest_discrete_uniform("subsample", 0.01, 1, 0.01),
            "colsample_bytree": trial.suggest_discrete_uniform("colsample_bytree", 0.01, 1, 0.01),
            "min_split_gain": trial.suggest_discrete_uniform("min_split_gain", 0.01, 1, 0.01),
            "min_child_samples": trial.suggest_int("min_child_samples", 1, 50),
            "n_jobs": trial.suggest_int("n_jobs", -1, -1)}
    return space
trials = 3 #liczba prób

def objective(trial, model, X, y):
    model_space = get_space(trial)

    mdl = model(**model_space)
    scores = cross_validate(mdl, X, y, groups=train_folds, scoring=scoring, cv=GroupKFold(n_splits=len(np.unique(train_folds))), return_train_score=True)

    return np.mean(scores['test_f1_macro'])

Liczba prób powinna być większa niż 60, ale ze względu na brak czasu nie będziemy jej zwiększać. W praktyce korzystając z pakietu LightGBM daje się liczbę prób rzędu kilkuset lub nawet więcej i definiuje się parametr early_stopping_round w którm podaje się, przez ile prób metryka walidacyjna nie ulega poprawie.

In [69]:
study = optuna.create_study(direction='maximize')

[I 2022-12-19 20:36:26,629] A new study created in memory with name: no-name-ff74dbf8-a3d0-440c-8df6-b45c0cc97212


In [70]:
%%time
study.optimize(lambda x: objective(x, model, X_train, y_train), n_trials=trials)
print('params: ', study.best_params)

C:\Users\jakub\AppData\Local\Temp\ipykernel_12420\54684782.py:7: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  "learning_rate": trial.suggest_discrete_uniform("learning_rate", 0.00001, 1, 0.0001),
c:\Users\jakub\AppData\Local\Programs\Python\Python310\lib\site-packages\optuna\distributions.py:668: UserWarning: The distribution is specified by [1e-05, 1] and step=0.0001, but the range is not divisible by `step`. It will be replaced by [1e-05, 0.99991].
  warnings.warn(
C:\Users\jakub\AppData\Local\Temp\ipykernel_12420\54684782.py:8: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  "subsample": trial.suggest_discrete_uniform("subsample",

params:  {'boosting_type': 'gbdt', 'num_leaves': 19, 'n_estimators': 167, 'max_depth': 167, 'learning_rate': 0.62661, 'subsample': 0.22, 'colsample_bytree': 0.03, 'min_split_gain': 0.19, 'min_child_samples': 40, 'n_jobs': -1}
CPU times: total: 2min 17s
Wall time: 1min 6s


In [71]:
#Wytrenuj model o najlepszych hiperparametrach, przeprowadź predykcję i wylicz metryki
params = study.best_params
print(params)
clf_both_best_LGBM = lgb.LGBMClassifier(random_state=42, **params)
clf_both_best_LGBM.fit(X_train_both, y_train)
both_preds_best = clf_both_best_LGBM.predict(X_test_both)
print(confusion_matrix(y_test, both_preds_best))
print(accuracy_score(y_test,both_preds_best))

{'boosting_type': 'gbdt', 'num_leaves': 19, 'n_estimators': 167, 'max_depth': 167, 'learning_rate': 0.62661, 'subsample': 0.22, 'colsample_bytree': 0.03, 'min_split_gain': 0.19, 'min_child_samples': 40, 'n_jobs': -1}
[[54  0 13  0  1 26  2  2  2]
 [ 1  3  5  0  1  2  0  1  3]
 [ 3  0 79  6  0  2  0  2  6]
 [ 0  0  8 50  0  0  0  1  9]
 [ 4  1  7  5 49  5  2  5  6]
 [ 9  0  1  0  2 65  4  0  8]
 [ 1  0  0  0 15  1 55  1  0]
 [ 4  1 25  2  0  0  0 44  1]
 [ 1  2  7  2  3  1  2  5 77]]
0.675177304964539


Porównaj uzyskane wszystkimi metodami macierze pomyłek. Która klasa jest najczęściej źle klasyfikowna, bez względu na użyty model? Jak myślisz, dlaczego?

Najczęściej elementy są błędnie klasyfikowane do ostatniego zbioru. Prawdopodobnie jest to spowodowane tym, że w zbiorze uczącym nie ma elementów tej klasy.